### Shared Word

In [30]:
import os
import multiprocessing
#import multiprocessing as mp
from multiprocessing import Process, Manager, Pool, Queue
from itertools import islice
import re
import pandas as pd
import numpy as np
import glob
import nltk
from nltk import word_tokenize
from functools import reduce
from pathlib import Path
import shutil

In [2]:
nprocs = multiprocessing.cpu_count()
print(f"Number of CPU cores: {nprocs}")

Number of CPU cores: 16


In [3]:
lang_folder = "English"  # Arabic, English, French, German, Turkish, Spanish, Portuguese, Dutch, Italian ==> target language
lang_pair = "Turkish"  # Arabic, English, French, German, Turkish, Spanish, Portuguese, Dutch, Italian ==> native language 

In [4]:
Path(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/Deployment/Result/4-Shared Word/{lang_folder.capitalize()} {lang_pair.capitalize()}").mkdir(parents=True, exist_ok=True)

In [5]:
def remove_repetition(word_group):
    words = word_tokenize(word_group)
    word_unique = set(words)
    if len(word_unique) == 1:
        return "repetitive_word_group"
    else:
        return word_group

In [6]:
lang_pair_list = glob.glob(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/Web Scrapping/Result/{lang_folder.capitalize()}/{lang_folder.capitalize()}_And_{lang_pair.lower().capitalize()}*_All.xlsx")
lang_pair_list

['/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/Web Scrapping/Result/English/English_And_Turkish (tur)_Etymologeek_Result_All.xlsx']

In [7]:
df_pair = pd.read_excel(f"{lang_pair_list[0]}")
df_pair

,dict_entry_main,turkish_word
0,school,okul
1,police,polis
2,cop,polis
3,policeman,polis
4,policewoman,polis
...,...,...
238,skeletal,iskelet
239,masa,masa
240,calorie,kalori
241,kismet,kısmet


In [8]:
disable_list = []

In [9]:
words = df_pair["dict_entry_main"].values.tolist()

In [10]:
disable_set = set(disable_list)
words_set = set(words)

In [11]:
word_list = list(words_set.difference(disable_set))

In [12]:
# 2 way  Not: 1 way is long and in Shared Word ReDe part 
data_kind_list = ["Two", "Three", "Four", "Five", "Sentence"]
for i in data_kind_list:
    sent_ngram_file = f"{i}" # Two, Three, Four, Five, Sentence
 
    if sent_ngram_file.lower() != "sentence":
        data_kind = f"{sent_ngram_file}gram"  # Twogram, Threegram, Fourgram, Fivegram
        folder_path = "N Gram"
    else:
        data_kind = "sentence"
        folder_path = "Sentence"

    df = pd.read_csv(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/Result/{folder_path.capitalize()}/Merge/{data_kind.capitalize()}_Merge.csv")
    d_list  = df.iloc[:,0].values.tolist()

    resultlist = []
    manager = multiprocessing.Manager()
    resultlist = manager.list()
    
    def word_in_wordgroup(d_list):
        mergelist = []
        try:
            word = d_list.split()
        except:
            pass
        var1 = range(len(word))
        for j in var1:
            if word[j] in word_list:
                mergelist.append(word[j])
                if len(mergelist) == len(word):
                        resultlist.append(d_list)
                            
    if __name__ == '__main__':
        # with Pool(16) as p:
        with Pool(nprocs) as p: # Pool number CPU sayısına eşit olursa tüm CPU lar çalışır
            p.map(word_in_wordgroup, d_list) # string_word liste

    result_list = list(resultlist)
    df_result = pd.DataFrame(result_list, columns=[0])  # add columns parameter for empty result
    df_result = df_result.rename(columns = {0: f"{data_kind.lower()}"})
    df_merge = pd.merge(df_result, df, how="left", on=f"{data_kind.lower()}")
    df_merge_result = df_merge.sort_values(by="frequency", ascending=False)
    df_merge_result.drop_duplicates(inplace=True)
    df_merge_result[f"{data_kind.lower()}"] = df_merge_result[f"{data_kind.lower()}"].apply(lambda x: remove_repetition(x))  # added remove_repetition 
    df_merge_result = df_merge_result[df_merge_result[f"{data_kind.lower()}"] != "repetitive_word_group"]  # remove_repetition part
    df_merge_result.reset_index(drop=True, inplace=True)

    df_merge_result.to_excel(f"{lang_folder.capitalize()}_{lang_pair.capitalize()}_Shared_{data_kind.capitalize()}.xlsx", index=False)
    

#### Concat Result

In [13]:
df_word = pd.DataFrame(word_list, columns=["word"])
df_word

,word
0,hotel
1,warranty
2,guitar
3,bravo
4,villa
...,...
238,barbarian
239,cop
240,dynamic
241,technique


In [14]:
df_shared_twogram = pd.read_excel(f"{lang_folder.capitalize()}_{lang_pair.capitalize()}_Shared_Twogram.xlsx")
df_shared_twogram = df_shared_twogram.loc[:,["twogram","frequency"]]
df_shared_twogram.rename(columns={"frequency":"freq_twogram"}, inplace=True)
df_shared_twogram

,twogram,freq_twogram
0,police chief,1154
1,police commissioner,1100
2,phone bill,761
3,catholic school,611
4,theme music,591
...,...,...
852,school german,4
853,admiral check,4
854,apartment hamburger,4
855,liberal school,4


In [15]:
df_shared_threegram = pd.read_excel(f"{lang_folder.capitalize()}_{lang_pair.capitalize()}_Shared_Threegram.xlsx")
df_shared_threegram = df_shared_threegram.loc[:,["threegram","frequency"]]
df_shared_threegram.rename(columns={"frequency":"freq_threegram"}, inplace=True)
df_shared_threegram

,threegram,freq_threegram
0,dollar dollar bill,54
1,chef chef check,19
2,chinese mafia system,14
3,electricity bill phone,13
4,cookie coffee tea,12
5,police chief chief,11
6,vampire vampire style,9
7,cheese coffee cake,9
8,tea coffee whisky,9
9,sugar coffee alcohol,7


In [16]:
df_shared_fourgram = pd.read_excel(f"{lang_folder.capitalize()}_{lang_pair.capitalize()}_Shared_Fourgram.xlsx")
df_shared_fourgram = df_shared_fourgram.loc[:,["fourgram","frequency"]]
df_shared_fourgram.rename(columns={"frequency":"freq_fourgram"}, inplace=True)
df_shared_fourgram

,fourgram,freq_fourgram
0,dollar dollar dollar bill,6
1,pasta pasta pasta cheese,4


In [17]:
df_shared_fivegram = pd.read_excel(f"{lang_folder.capitalize()}_{lang_pair.capitalize()}_Shared_Fivegram.xlsx")
df_shared_fivegram = df_shared_fivegram.loc[:,["fivegram","frequency"]]
df_shared_fivegram.rename(columns={"frequency":"freq_fivegram"}, inplace=True)
df_shared_fivegram

,fivegram,freq_fivegram
0,pasta pasta pasta pasta cheese,4


In [18]:
df_shared_sentence = pd.read_excel(f"{lang_folder.capitalize()}_{lang_pair.capitalize()}_Shared_Sentence.xlsx")
df_shared_sentence = df_shared_sentence.loc[:,["sentence","frequency"]]
df_shared_sentence.rename(columns={"frequency":"freq_sentence"}, inplace=True)
df_shared_sentence

,sentence,freq_sentence
0,merci madame,213
1,theme music,198
2,catholic school,122
3,tea coffee,103
4,coffee tea,91
...,...,...
145,cake boss,4
146,coffee boss,4
147,police telephone,4
148,check chief,4


In [19]:
df_shared_all = pd.concat([df_word, df_shared_twogram, df_shared_threegram, df_shared_fourgram, df_shared_fivegram, df_shared_sentence], axis=1)
df_shared_all

,word,twogram,freq_twogram,threegram,freq_threegram,fourgram,freq_fourgram,fivegram,freq_fivegram,sentence,freq_sentence
0,hotel,police chief,1154,dollar dollar bill,54.0,dollar dollar dollar bill,6.0,pasta pasta pasta pasta cheese,4.0,merci madame,213.0
1,warranty,police commissioner,1100,chef chef check,19.0,pasta pasta pasta cheese,4.0,NaN,NaN,theme music,198.0
2,guitar,phone bill,761,chinese mafia system,14.0,NaN,NaN,NaN,NaN,catholic school,122.0
3,bravo,catholic school,611,electricity bill phone,13.0,NaN,NaN,NaN,NaN,tea coffee,103.0
4,villa,theme music,591,cookie coffee tea,12.0,NaN,NaN,NaN,NaN,coffee tea,91.0
...,...,...,...,...,...,...,...,...,...,...,...
852,NaN,school german,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
853,NaN,admiral check,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
854,NaN,apartment hamburger,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
855,NaN,liberal school,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [20]:
df_shared_all.to_excel(f"{lang_folder.capitalize()}_{lang_pair.capitalize()}_Shared_Result_With_Frequency.xlsx", index=False)

In [21]:
def word_in_twogram(df, list_column, target_column):

    '''word_in_twogram(df, "word", "twogram"):
       df is dataframe, list_column and target_column are 
       dataframe column string name. list_column convert list
       values that are in target column.
    '''
    
    df_select = df[[f"{target_column}"]].dropna()
    df_word_result = pd.DataFrame()
    for i in df[f"{list_column}"].dropna():
        try:
            word_in_twogram = df_select[df_select[f"{target_column}"].str.contains(fr"(?:\s|^){i}(?:\s|$)", na=True)] 
        except:
            pass        
        word_in_twogram.insert(0,"word",i)
        df_word_result = pd.concat([df_word_result,word_in_twogram], axis=0)
    df_word_result.reset_index(drop=True, inplace=True)

    return df_word_result

In [22]:
df_word_order_twogram = word_in_twogram(df_shared_all, "word", "twogram")
df_word_order_threegram = word_in_twogram(df_shared_all, "word", "threegram") 
df_word_order_fourgram = word_in_twogram(df_shared_all, "word", "fourgram") 
df_word_order_fivegram = word_in_twogram(df_shared_all, "word", "fivegram")
df_word_order_sentence = word_in_twogram(df_shared_all, "word", "sentence") 

In [23]:
df_word_order_join_twogram = df_word_order_twogram.groupby(["word"])["twogram"].apply(", ".join).reset_index()   # df_word_order_11.groupby(["word"])[f"twogram_pair_{lang_pair.lower()}"].transform(lambda x: ','.join(x))
df_word_order_join_threegram = df_word_order_threegram.groupby(["word"])["threegram"].apply(", ".join).reset_index()
df_word_order_join_fourgram = df_word_order_fourgram.groupby(["word"])["fourgram"].apply(", ".join).reset_index()
df_word_order_join_fivegram = df_word_order_fivegram.groupby(["word"])["fivegram"].apply(", ".join).reset_index()
df_word_order_join_sentence = df_word_order_sentence.groupby(["word"])["sentence"].apply(", ".join).reset_index()

In [24]:
dfs = [df_word_order_join_twogram,df_word_order_join_threegram,df_word_order_join_fourgram,df_word_order_join_fivegram,df_word_order_join_sentence]

In [25]:
df_word_order_join_all = reduce(lambda  left,right: pd.merge(left,right, on=['word'], how='outer'), dfs)  # left,right make left to right merge
#df_word_order_join_all = reduce(lambda  right,left: pd.merge(left,right, on=['word'], how='outer'), dfs)  # right,left make right to left merge
df_word_order_join_all  

,word,twogram,threegram,fourgram,fivegram,sentence
0,acid,acid factor,NaN,NaN,NaN,NaN
1,active,"active volcano, system active, active system, ...",NaN,NaN,NaN,active volcano
2,admiral,"bill admiral, channel admiral, office admiral,...",NaN,NaN,NaN,NaN
3,admiralty,german admiralty,NaN,NaN,NaN,NaN
4,afghan,"afghan group, afghan terrorist",afghan terrorist group,NaN,NaN,NaN
...,...,...,...,...,...,...
193,warden,"chief warden, check warden, coffee warden, cig...",NaN,NaN,NaN,"chief warden, cigar warden"
194,whiskey,"japanese whiskey, whiskey saloon, whiskey brav...",NaN,NaN,NaN,japanese whiskey
195,whisky,"music whisky, whisky cognac, whisky saloon, co...",tea coffee whisky,NaN,NaN,"whisky cognac, cognac whisky, tea coffee whisky"
196,yoghurt,yoghurt coffee,NaN,NaN,NaN,NaN


In [26]:
df_word_order_join_all.to_excel(f"{lang_folder.capitalize()}_{lang_pair.capitalize()}_Shared_Join_Result_Without_Frequency.xlsx", index=False)

#### Copy Move And Delete

In [28]:
output_file = glob.glob(f"{lang_folder.capitalize()}_{lang_pair.capitalize()}_Shared*.xlsx")
output_file

['English_Turkish_Shared_Twogram.xlsx',
 'English_Turkish_Shared_Threegram.xlsx',
 'English_Turkish_Shared_Fourgram.xlsx',
 'English_Turkish_Shared_Fivegram.xlsx',
 'English_Turkish_Shared_Sentence.xlsx',
 'English_Turkish_Shared_Result_With_Frequency.xlsx',
 'English_Turkish_Shared_Join_Result_Without_Frequency.xlsx']

In [31]:
for k in output_file:
    source = k # source directory
    destination = f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/Deployment/Result/4-Shared Word/{lang_folder.capitalize()} {lang_pair.capitalize()}"
    shutil.copy2(source, destination)

In [32]:
for i in output_file:
    try:
        os.remove(i)
    except:
        pass